In [1]:
import nltk
from os import getcwd
import w1_unittest

nltk.download('twitter_samples')
nltk.download('stopwords')

[nltk_data] Downloading package twitter_samples to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
filePath = f"{getcwd()}/../tmp2/"
nltk.data.path.append(filePath)

In [3]:
import numpy as np
import pandas as pd
from nltk.corpus import twitter_samples 

from utils import process_tweet, build_freqs

In [4]:
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

In [5]:
test_pos = all_positive_tweets[4000:]
train_pos = all_positive_tweets[:4000]
test_neg = all_negative_tweets[4000:]
train_neg = all_negative_tweets[:4000]

train_x = train_pos + train_neg 
test_x = test_pos + test_neg

In [6]:
train_y = np.append(np.ones((len(train_pos), 1)), np.zeros((len(train_neg), 1)), axis=0)
test_y = np.append(np.ones((len(test_pos), 1)), np.zeros((len(test_neg), 1)), axis=0)

In [7]:
print("train_y.shape = " + str(train_y.shape))
print("test_y.shape = " + str(test_y.shape))

train_y.shape = (8000, 1)
test_y.shape = (2000, 1)


In [8]:
freqs = build_freqs(train_x, train_y)

print("len(freqs) = " + str(len(freqs.keys())))

len(freqs) = 11436


In [9]:
print('a positive tweet: \n', train_x[0])
print('\processed version of the tweet: \n', process_tweet(train_x[0]))

a positive tweet: 
 #FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)
\processed version of the tweet: 
 ['followfriday', 'top', 'engag', 'member', 'commun', 'week', ':)']


In [10]:
def sigmoid(z): 
    
    h = 1/(1+np.exp(-z)) 
    
    return h

In [11]:
def gradientDescent(x, y, theta, alpha, num_iters):
    
    m = x.shape[0]
    
    for i in range(0, num_iters):

        z = np.dot(x,theta)

        h = sigmoid(z)

        J = -(1/m)*(np.dot(y.T, np.log (h)) + np.dot((1-y).T , np.log(1-h)))

        theta = theta-(alpha/m)* np.dot(x.T,(h-y))

    J = float(J)
    return J, theta

In [12]:
def extract_features(tweet, freqs, process_tweet=process_tweet):
    
    word_l = process_tweet(tweet)
    x = np.zeros(3) 
    x[0] = 1 

    for word in word_l:

        x[1] +=  freqs.get((word, 1), 0)

        x[2] +=  freqs.get((word, 0), 0)
        
    
    x = x[None, :]  
    assert(x.shape == (1, 3))
    return x

In [13]:
#train

X = np.zeros((len(train_x), 3))
for i in range(len(train_x)):
    X[i, :]= extract_features(train_x[i], freqs)

Y = train_y

J, theta = gradientDescent(X, Y, np.zeros((3, 1)), 1e-9, 1500)
print(f"The cost after training is {J:.8f}.")
print(f"The resulting vector of weights is {[round(t, 8) for t in np.squeeze(theta)]}")

The cost after training is 0.22522315.
The resulting vector of weights is [6e-08, 0.00053818, -0.0005583]


In [14]:
def predict_tweet(tweet, freqs, theta):
    
    x = extract_features(tweet, freqs)
    y_pred = sigmoid(np.dot(x,theta))
    
    return y_pred

In [15]:
def test_logistic_regression(test_x, y_hat, freqs, theta, predict_tweet=predict_tweet):
    y_hat = []
    
    for tweet in test_x:
        y_pred = predict_tweet(tweet, freqs, theta)
        
        if y_pred > 0.5:
            y_hat.append(1.0)
        else:
            y_hat.append(0.0)

    y_hat = np.array(y_hat)
    m = test_y.shape[0]
    y_hat = y_hat.reshape(m, 1)
    print(y_hat.shape)
    accuracy = np.sum(y_hat == test_y )/m
    
    return accuracy

In [16]:
tmp_accuracy = test_logistic_regression(test_x, test_y, freqs, theta)
print(f"Logistic regression model's accuracy = {tmp_accuracy:.4f}")

(2000, 1)
Logistic regression model's accuracy = 0.9950


In [17]:

my_tweet = 'This is a ridiculously bright movie. The plot was terrible and I was sad until the ending!'
print(process_tweet(my_tweet))
y_hat = predict_tweet(my_tweet, freqs, theta)
print(y_hat)

if y_hat > 0.5:
    print('Positive sentiment')
else: 
    print('Negative sentiment')

['ridicul', 'bright', 'movi', 'plot', 'terribl', 'sad', 'end']
[[0.48125423]]
Negative sentiment
